<a href="https://colab.research.google.com/github/PokeMaster16/My-first-upload-/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
from sklearn.preprocessing import StandardScaler


#label_encoders = {}
#for column in ['sex', 'smoker','region']:
#le = LabelEncoder()
#dataset[column] = le.fit_transform(dataset[column])
#label_encoders[column] = le
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaled = scaler.fit_transform(dataset.drop('expenses', axis=1))

x = pd.DataFrame(x_scaled, columns=dataset.drop('expenses', axis=1).columns)
y = dataset['expenses']


from sklearn.model_selection import train_test_split

x = dataset.drop('expenses', axis=1)
y = dataset['expenses']

train_dataset, test_dataset, train_labels, test_labels = train_test_split(x, y, test_size=0.2,
                                                                          random_state=42)
model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=[train_dataset.shape[1]]),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


# Compile the model
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae','mse']
)

# Train the model
history = model.fit(
    train_dataset,
    train_labels,
    epochs=100,
    validation_split=0.2,
    verbose=0  # Set to 1 to see training output
)

loss, mae,mse = model.evaluate(test_dataset, test_labels, verbose=2)
print(f"Mean Absolute Error on test data: ${mse:.2f}")


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
